In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train_eval import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [2]:
def train_eval_light():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model = get_model(config)
    model, runtime = train_models([model], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



In [3]:
# See what keys are in the h5py data file
get_hdf_keys(dir_path)

({'1_county.h5': ['Los_Angeles', 'New_York', 'Sacramento'],
  '2_town.h5': ['town_0', 'town_1', 'town_2'],
  '3_village.h5': ['village_0', 'village_1', 'village_2'],
  '4_neighborhood.h5': ['germany'],
  '5_household.h5': ['household_0', 'household_1', 'household_2'],
  '6_apartment.h5': ['apartment_0', 'apartment_1', 'apartment_2']},
 {'1_county.h5': ['60min'],
  '2_town.h5': ['15min', '60min'],
  '3_village.h5': ['15min', '60min'],
  '4_neighborhood.h5': ['15min', '60min'],
  '5_household.h5': ['15min', '60min'],
  '6_apartment.h5': ['15min', '5min', '60min']})

In [4]:
# run parameters

sweeps = 2

scale_location_pairs = (
    ('1_county', 'Sacramento'),
    #('2_town', 'town_0'),
    #('3_village', 'village_0'),
    #('4_neighborhood', 'germany'),
    #('5_household', 'household_0'),
      )



models = [
        # 'rf',
        # 'xgb', 
        # 'gru', 
        # 'lgbm',  
        #'nbeats',
        #'transformer',
        #'tft'
        ]

for scale, location in scale_location_pairs:

    for model in models:
        # place holder initialization of config file (will be updated in train_eval_light()
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': False,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
        }

        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="WattCast_tuning")
        wandb.agent(sweep_id, train_eval_light, count=sweeps)


Create sweep with ID: 6yg0otu7
Sweep URL: https://wandb.ai/wattcast/Wattcast_tuning/sweeps/6yg0otu7


wandb: Agent Starting Run: o9gv885f with config:
wandb: 	batch_size: 32
wandb: 	d_model: 128
wandb: 	datetime_encodings: 0
wandb: 	dropout: 0.1
wandb: 	lr: 0.1
wandb: 	nhead: 4
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'datetime_encodings' was locked by 'sweep' (ignored update).


Getting data...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Evaluating model...
Predicting test set...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), us

2017-08-16 23:00:00 2017-08-17 00:00:00
2017-08-17 23:00:00 2017-08-18 00:00:00
2017-08-18 23:00:00 2017-08-19 00:00:00
2017-08-19 23:00:00 2017-08-20 00:00:00
2017-08-20 23:00:00 2017-08-21 00:00:00
2017-08-21 23:00:00 2017-08-22 00:00:00
2017-08-22 23:00:00 2017-08-23 00:00:00
2017-08-23 23:00:00 2017-08-24 00:00:00
2017-08-24 23:00:00 2017-08-25 00:00:00
2017-08-25 23:00:00 2017-08-26 00:00:00
2017-08-26 23:00:00 2017-08-27 00:00:00
2017-08-27 23:00:00 2017-08-28 00:00:00
2017-08-28 23:00:00 2017-08-29 00:00:00
2017-08-29 23:00:00 2017-08-30 00:00:00
2017-08-30 23:00:00 2017-08-31 00:00:00
2017-08-31 23:00:00 2017-09-01 00:00:00
2017-09-01 23:00:00 2017-09-02 00:00:00
2017-09-02 23:00:00 2017-09-03 00:00:00
2017-09-03 23:00:00 2017-09-04 00:00:00
2017-09-04 23:00:00 2017-09-05 00:00:00
2017-09-05 23:00:00 2017-09-06 00:00:00
2017-09-06 23:00:00 2017-09-07 00:00:00
2017-09-07 23:00:00 2017-09-08 00:00:00
2017-09-08 23:00:00 2017-09-09 00:00:00
2017-09-09 23:00:00 2017-09-10 00:00:00


eval_loss,▁
eval_loss,0.29536
